In [67]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from collections import Counter
# from config import *
import json
import datetime
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel



In [68]:
#This function will clean our text from data that is not important so that has no weight 
def clean_text(tweet):
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
   
    tweet = tweet.lower() # Transform in lowercase

    tweet = re.sub(r'@[a-zA-Z]+', '', tweet) # Here we remove the mentions in the tweet ex: @canodep
    tweet = re.sub(r"\B#([a-z0-9]{2,})(?![~!@#$%^&*()=+_`\-\|\/'\[\]\{\}]|[?.,]*\w)", '', tweet) # Here we remove the hashtags, because we will treat it later
    tweet = re.sub(r'[^\w\s]', '', tweet) # Here we remove punctuation marks
    tweet = re.sub(r'http\S+', '',tweet) # Remove http and https
    tweet = tweet.split() # Tokenize the text to get a list of terms

    tweet = [word for word in tweet if word not in stop_words]  # eliminate the stopwords
    tweet = [stemmer.stem(word) for word in tweet] # Perform stemming 
    return tweet
    

In [69]:
docs_path = 'data/tw_hurricane_data.json'
tweets_title = 'data/tweet_document_ids_map.csv'

tweets_id_title = {}

with open(tweets_title) as fp:
    lines = fp.readlines()


for l in lines:
    l = l.strip().split("\t")
    tweets_id_title[int(l[1])] =  l[0]


tweets = []
lines = []

for line in open(docs_path, 'r'):
    lines.append(line)
    #media = json.loads(line).get('entities').get('media')
    tweets.append({
        'id' : int(json.loads(line).get('id')),
        'title' : tweets_id_title[int(json.loads(line).get('id'))],
        'text': json.loads(line).get('full_text'),
        'username' : json.loads(line).get('user').get('screen_name'),
        'date' : json.loads(line).get('created_at'),
        'hashtag' : list(map(lambda hashtag:  hashtag.get('text'),  json.loads(line).get('entities').get('hashtags'))),
        'like' : json.loads(line).get('favorite_count'),
        'rt' : json.loads(line).get('retweet_count'),
        'URL' : 'https://twitter.com/' + json.loads(line).get('user').get('screen_name') + "/status/" + str(json.loads(line).get('id'))
    }) 


In [71]:
tweets_text = list(map(lambda tweet: tweet.get('text'), tweets))
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(tweets_text)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()

tfidf = pd.DataFrame(denselist, columns=feature_names)


cosine_sim = linear_kernel(tfidf, tfidf)
print(cosine_sim)

/home/alfa/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[[1.         0.03537872 0.00894123 ... 0.04613254 0.00698248 0.01616436]
 [0.03537872 1.         0.00262344 ... 0.00225951 0.00204873 0.00157583]
 [0.00894123 0.00262344 1.         ... 0.01204297 0.01091949 0.00839899]
 ...
 [0.04613254 0.00225951 0.01204297 ... 1.         0.00940472 0.02576522]
 [0.00698248 0.00204873 0.01091949 ... 0.00940472 1.         0.00655903]
 [0.01616436 0.00157583 0.00839899 ... 0.02576522 0.00655903 1.        ]]
